In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
import mpl_toolkits
from scipy.stats import loguniform
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
%matplotlib inline

In [2]:
train = pd.read_csv('engineered_data.csv')

In [3]:
param_distributions = {
    #"n_estimators": [5, 10, 20, 50, 100, 200, 300, 400, 500, 600,650, 700, 800, 900, 1000],
    #"max_leaf_nodes": [2, 3, 4, 5, 10, 20, 50, 100, 150, 200, 250],
    #"learning_rate": [1, 2]
    #"max_features": range(7,19,2)
    #"min_samples_leaf": range(1, 12, 2)
    "subsample": [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
}
br = GradientBoostingRegressor(learning_rate=0.1, min_samples_split=10, max_depth=4, n_estimators=2100, max_leaf_nodes= 50, random_state=0, max_features=15)
search_cv = GridSearchCV(estimator=br, 
                   param_grid=param_distributions,
                   scoring = 'neg_mean_squared_error', 
                   verbose=1,
                   error_score='raise',
                   n_jobs=4)

target = "price"
columns = train.columns.tolist()
columns = [c for c in columns if c not in ["price"]]
search_cv.fit(train[columns], train[target])


print("Best parameters:", search_cv.best_params_)
print("Lowest RMSE: ", (-search_cv.best_score_)**(1/2.0))

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best parameters: {'subsample': 0.8}
Lowest RMSE:  74703.75080060621


In [ ]:
from sklearn.metrics import mean_squared_error

br = GradientBoostingRegressor(learning_rate=0.1, min_samples_split=10, max_depth=4, n_estimators=2100, max_leaf_nodes= 50, random_state=0, max_features=15)
training = train.sample(frac=0.7, random_state=5)
test = train.loc[~train.index.isin(training.index)]
br.fit(train[columns], train[target])
clf = br.predict(test[columns])
# Compute error between our test predictions and the actual values.
lin_mse = mean_squared_error(clf, test[target], squared=False)
print("Computed error:", lin_mse)